---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    # if max degree above 80 PA
    # elif CC above 0.1 SW_L
    # else SW_H
    return ['PA','SW_L','SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')
 
#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [13]:
def salary_predictions():
    def manage(node):
        man_sal = node[1]['ManagementSalary']
        if man_sal == 0:
            return 0
        elif man_sal == 1:
            return 1
        else:
            return None
    from sklearn.naive_bayes import GaussianNB
    
    df = pd.DataFrame(index = G.nodes())
    df['pr'] = pd.Series(nx.pagerank(G))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(nx.degree(G))
    df['b/wCent'] = pd.Series(nx.betweenness_centrality(G))
    df['cloCent'] = pd.Series(nx.closeness_centrality(G))
    df['degCent'] = pd.Series(nx.degree_centrality(G))
    df['is_manage'] = pd.Series([ manage(node) for node in G.nodes(data=True)])
    
    df_train = df[~pd.isnull(df['is_manage'])]
    df_test = df[pd.isnull(df['is_manage'])]
    
    feat = ['pr','clustering','degree','b/wCent','cloCent','degCent']
    X_train = df_train[feat]
    Y_train = df_train['is_manage']
    
    X_test = df_test[feat]
    
    model = GaussianNB().fit(X_train,Y_train)
    
    prediction = model.predict_proba(X_test)[:,1]
    
    
    return pd.Series(prediction, X_test.index)
salary_predictions()

1       0.023378
2       0.999993
5       1.000000
8       0.003611
14      0.960733
18      0.156274
27      0.313092
30      0.320690
31      0.014667
34      0.000611
37      0.001695
40      0.059623
45      0.018314
54      0.015534
55      0.013660
60      0.021237
62      1.000000
65      1.000000
77      0.001239
79      0.010947
97      0.000557
101     0.001276
103     0.028653
108     0.009555
113     0.992205
122     0.000499
141     0.998761
142     1.000000
144     0.000403
145     0.729898
          ...   
913     0.000282
914     0.000357
915     0.000001
918     0.000581
923     0.000005
926     0.000120
931     0.000238
934     0.000061
939     0.000023
944     0.000003
945     0.000015
947     0.000009
950     0.012967
951     0.000027
953     0.000213
959     0.000009
962     0.000002
963     0.007195
968     0.000002
969     0.000003
974     0.000050
984     0.000045
987     0.000059
989     0.000003
991     0.000003
992     0.000026
994     0.000013
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [14]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [27]:
def new_connections_predictions():
    from sklearn.naive_bayes import GaussianNB
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
        
    pref_att = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index  = [ (x[0],x[1]) for x in pref_att ])
    df['pref_att'] = [ x[2] for x in pref_att ]
    
    cn_sh = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_sh = pd.DataFrame(index = [ (x[0],x[1]) for x in cn_sh ])
    df_cn_sh['cn_sh'] = [ x[2] for x in cn_sh ]
    df = df.join(df_cn_sh,how = 'outer')
    df['cn_sh'] =df['cn_sh'].fillna(value = 0)
    
    jac_coeff = list(nx.jaccard_coefficient(G))
    #df_jac = pd.DataFrame(index = [ (x[0],x[1]) for x in jac_coeff ])
    df['jac_coeff'] = [ x[2] for x in jac_coeff ]
    
    df['rai'] = [ x[2] for x in list(nx.resource_allocation_index(G)) ]
    
    df = future_connections.join(df,how='outer')
    
    feat = ['pref_att','cn_sh','jac_coeff','rai']
    
    
    df_train = df[ ~pd.isnull(df['Future Connection']) ]
    df_test =  df[  pd.isnull(df['Future Connection']) ]
    
    X_train = df_train[feat]
    Y_train = df_train['Future Connection']
    X_test = df_test[feat]
    
    model = GaussianNB().fit(X_train,Y_train)
    
    prediction = model.predict_proba(X_test)[:,1]
    
    return pd.Series(prediction, X_test.index)
new_connections_predictions()

(0, 9)          0.000906
(0, 19)         0.050153
(0, 20)         0.915791
(0, 35)         0.001277
(0, 38)         0.000534
(0, 42)         0.999999
(0, 43)         0.000247
(0, 50)         0.000201
(0, 53)         0.983663
(0, 54)         0.003211
(0, 62)         1.000000
(0, 63)         0.975366
(0, 69)         0.002785
(0, 72)         0.000195
(0, 83)         0.999995
(0, 90)         0.000502
(0, 91)         0.000311
(0, 95)         0.032793
(0, 100)        0.000924
(0, 106)        1.000000
(0, 108)        0.000492
(0, 109)        0.000199
(0, 110)        0.000195
(0, 118)        0.000277
(0, 122)        0.000512
(0, 131)        0.005880
(0, 133)        1.000000
(0, 140)        0.004437
(0, 149)        0.052512
(0, 151)        0.000506
                  ...   
(988, 989)      0.000210
(988, 996)      0.000210
(988, 997)      0.181333
(988, 1000)     0.000210
(988, 1002)     0.000210
(989, 994)      0.000210
(989, 996)      0.000210
(989, 1003)     0.000210
(989, 1004)     0.000210
